In [ ]:
import pandas as pd
from unidecode import unidecode

Reading the PNAD data

In [ ]:
df = pd.read_csv(r"\repasse_prefeituras.csv",
                 encoding='Latin1')

Removing special characters, blank spaces, capital letters etc 

In [ ]:
df['alimentos'] = df['Descrição do Item'].str.replace(" \ ","").str.replace("_","").str.replace("9","").str.replace("8","").str.replace("7","").str.replace("6","").str.replace("5","").str.replace("4","").str.replace("3","").str.replace("2","").str.replace("1","").str.replace("0","").str.lower().str.replace("*","").str.replace(" ","").str.replace("(","").str.replace("kg","").str.replace(")","").str.replace("/","").str.replace(",","").str.replace(".","").str.replace("-","").str.replace(":","").apply(lambda s:unidecode(s) if type(s) == str else s).astype('str')

Reading the food's classification data

In [ ]:
clas_alm = pd.read_excel(r'\Warning labels food items_nova.xlsx',engine='openpyxl')

Removing special characters, blank spaces, capital letters etc 

In [ ]:
clas_alm['alimento'] = clas_alm['DESCRIÇÃO DO ALIMENTO'].str.replace(" \ ","").str.replace("_","").str.replace("9","").str.replace("8","").str.replace("7","").str.replace("6","").str.replace("5","").str.replace("4","").str.replace("3","").str.replace("2","").str.replace("1","").str.replace("0","").str.lower().str.replace("*","").str.replace(" ","").str.replace("(","").str.replace("kg","").str.replace(")","").str.replace("/","").str.replace(",","").str.replace(".","").str.replace("-","").str.replace(":","").apply(lambda s:unidecode(s) if type(s) == str else s).astype('str')

Selecting the columns that we will use

In [ ]:
clasf = clas_alm
clasf = clasf[['DESCRIÇÃO DO ALIMENTO','alimento','nova']]

Merging the data

In [ ]:
tt = pd.merge(df,clasf,how='left',left_on=['alimentos'],right_on=['alimento'])

Selecting the unclassified data 

In [ ]:
teste2 = tt[tt['alimento'].isna()]
n_clasf = pd.Series(teste2['alimentos'].unique())

alimentos = clasf['alimento'].to_list()

In this for loop we test in each (non classified) food if we have some correspondence in the food's classification data. 
For example, if the word 'abacateceasa' was not classified in the first merge because we don't have an equal word in 
the food's classification data, in this for loop we will obtain the word 'abacate', allowing the classification of the 
entire word 'abacateceasa'. 

In [ ]:
dic = {}
for i in n_clasf:
    ls = []
    for j in alimentos:
        if j in i:
            ls.append(j)    
    dic[i] = ls

In this for loop we remove repetitions (using the set())

In [ ]:
dic_new = {}
for x,v in dic.items():
    dic_new[x] = list(set(v))
teste3 = pd.DataFrame.from_dict(dic_new,orient='index')

In this for loop we obtain the nova's number of each new classified food 

In [ ]:
colunas = {}
for j in teste3.columns:
    colunas[j] = pd.merge(teste3,clasf,how='left',left_on=[j],right_on=['alimento'])[['nova']]
colunas = pd.concat(colunas,axis=1)

Merging the data  

In [ ]:
teste4 = pd.concat([teste3.reset_index(),colunas],axis=1).set_index('index')
teste4 = teste4.iloc[:,6:]
teste4['valor_unico'] = teste4.nunique(axis=1)

Selecting the data that still without any classification 

In [ ]:
teste5 = teste4[teste4['valor_unico'] == 0]

Selecting the data that have a coherent classification, with only one nova's number.
For example, if in the first for loop we identified the words 'torta' and 'morango'
for 'tortademorango' and they have differents classification (nova) number, we don't consider
'tortademorango' as classified. 

In [ ]:
teste6 = teste4[teste4['valor_unico'] == 1]